<a href="https://colab.research.google.com/github/TamannaAhmad/MenstrualCycleDietExercise/blob/main/FlowWellModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload the kaggle.json file to Google Colab

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaishnavidayanand02","key":"c5aac0a588367b4340c16201949b3ab2"}'}

Upload the kaggle.json file to Google Colab

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Install the Kaggle API client

In [ ]:
!pip install kaggle

Download the dataset using the Kaggle API

In [ ]:
!kaggle datasets download -d manasrai1406/menstrual-health-and-pcod-risk-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/manasrai1406/menstrual-health-and-pcod-risk-detection-dataset
License(s): MIT
  0% 0.00/1.28k [00:00<?, ?B/s]
100% 1.28k/1.28k [00:00<00:00, 1.94MB/s]


Unzip the dataset downloaded

In [ ]:
!kaggle datasets download -d manasrai1406/menstrual-health-and-pcod-risk-detection-dataset
!unzip menstrual-health-and-pcod-risk-detection-dataset.zip -d ./dataset

Dataset URL: https://www.kaggle.com/datasets/manasrai1406/menstrual-health-and-pcod-risk-detection-dataset
License(s): MIT
menstrual-health-and-pcod-risk-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  menstrual-health-and-pcod-risk-detection-dataset.zip
  inflating: ./dataset/period - Copy.csv  


In [ ]:
import numpy as np
import pandas as pd

data_path = '/content/dataset/period - Copy.csv'
df = pd.read_csv(data_path)

In [ ]:
df.columns

Index(['number_of_peak', 'Age', 'Length_of_cycle',
       'Estimated_day_of_ovulution', 'Length_of_Leutal_Phase',
       'Length_of_menses', 'Unusual_Bleeding', 'Weight', 'BMI',
       'Mean_of_length_of_cycle', 'Menses_score'],
      dtype='object')

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df["Unusual_Bleeding"]=df["Unusual_Bleeding"].map({"NO":0,"No":0,"no":0,"yes":1})
df["Menses_score"]=df["Menses_score"].astype(str).map({"1.0":0, "2.0":1, "3.0":2, "4.0":3, "4.2":4, "4.5":5, "5.0":6})

In [ ]:
df.drop("Income", axis=1, inplace=True)
df.drop("Height", axis=1, inplace=True)

In [ ]:
x=df.drop("Menses_score", axis=1)
y=df["Menses_score"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
dtrain = xgb.DMatrix(x_train,label = y_train)

In [ ]:
dtest = xgb.DMatrix(x_test,label = y_test)

In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss'
)
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print(y_pred)
df.columns

[1 0 2 1 1 1 6 0 1 2 1 2 1 2 3 2 6 3 3 2 1 3 3 2 2 3 2 3 2 3 6 2 6]


Index(['number_of_peak', 'Age', 'Length_of_cycle',
       'Estimated_day_of_ovulution', 'Length_of_Leutal_Phase',
       'Length_of_menses', 'Unusual_Bleeding', 'Weight', 'BMI',
       'Mean_of_length_of_cycle', 'Menses_score'],
      dtype='object')

In [27]:
ac=accuracy_score(y_test,y_pred)
ac

0.8787878787878788

In [28]:
model.save_model('model.json')